In [1]:
import torch
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from skimage import io
%matplotlib inline

In [2]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
        

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_output=4):
        super(ResNet,self).__init__()
        self.in_planes = 64
        
        self.conv1 = nn.Conv2d(3,64,kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #self.linear = nn.Linear(512*block.expansion, num_output) 
        self.linear = nn.Linear(153600, num_output)     

        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])
def image_loader(image_name):
    image = Image.open(image_name).convert('RGB')
    return image
transform = transforms.ToTensor()

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
start_epoch = 0
net = ResNet18().to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [6]:
class GateDataset(Dataset):
    def __init__(self, txt, transform, loader):
        f=open(txt, 'r')
        data = []
        count = 0
        for line in f:
            line = line.strip('\n')
            line = line.rstrip()
            words = line.split()
            label = np.array([float(words[1]),float(words[2]),float(words[3]),float(words[4])])
            label = torch.from_numpy(label)
            if count%10==0:
                data.append((words[0],label))
            count+=1
        self.data = data
        self.transform = transform
        self.loader = loader
        
    
    def __getitem__(self, index):
        img_file, label = self.data[index]
        img_file = './gate_img/'+img_file
        img = self.loader(img_file)
        if self.transform is not None:
            img = self.transform(img)
        return img, label
    
    def __len__(self):
        return len(self.data)   

In [7]:
train_data = GateDataset(txt='./gate_img/ground_truth.txt',transform=transforms.ToTensor(),loader=image_loader)
data_loader = DataLoader(train_data, batch_size=2,shuffle=True)


In [8]:
# num_block = [2,2,2,2]
# global in_planes
# in_planes = 64
# def _make_layer(block, planes, num_blocks, stride):
#     strides = [stride] + [1]*(num_blocks-1)
#     layers = []
#     global in_planes
#     for stride in strides:
#         layers.append(block(in_planes, planes, stride))
#         in_planes = planes * block.expansion
#     return nn.Sequential(*layers)



# print(len(data_loader))
# images, labels = next(iter(data_loader))
# print(images.size())
# conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
# bn1 = nn.BatchNorm2d(64)
# out = F.relu(bn1(conv1(images)))
# layer1 = _make_layer(BasicBlock,64,num_block[0],stride=1)

# out = layer1(out)
# print(out.size())
# layer2 = _make_layer(BasicBlock,128,num_block[1],stride=2)
# out = layer2(out)
# print(out.size())
# layer3 = _make_layer(BasicBlock,256,num_block[2],stride=2)
# out = layer3(out)
# print(out.size())
# layer4 = _make_layer(BasicBlock,512,num_block[3],stride=2)
# out = layer4(out)
# print(out.size())
# out = F.avg_pool2d(out, 4)
# print(out.size())
# out = out.view(out.size(0), -1)
# print(out.size())
# linear = nn.Linear(153600, 4)
# out = linear(out)
# print(out.size())






In [15]:
#optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9, weight_decay=5e-4)
#scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 150], gamma=0.1, last_epoch=-1)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.SmoothL1Loss()

In [14]:
def train(epoch, net):
    #print('\nEpoch: {} ==> lr: {}'.format(epoch, scheduler.get_last_lr()))
    net.train()
    train_loss = 0
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        #inputs, targets = inputs.to(device), targets.to(device)
        inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
        targets = torch.tensor(targets, dtype=torch.float32).to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        print(loss.item())
        

In [16]:
print('==> Training parameters:')
print('        start_epoch = {}'.format(start_epoch+1))
print('        lr @epoch=0 = {}'.format(0.1))
print('==> Starting training...')
for epoch in range(0, 5):
    if epoch>start_epoch:
        train(epoch, net)
    #scheduler.step()

==> Training parameters:
        start_epoch = 1
        lr @epoch=0 = 0.1
==> Starting training...


/home/colin/anaconda3/envs/deep_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/colin/anaconda3/envs/deep_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


1.2352497577667236
0.6581450700759888
0.4809007942676544
1.5059890747070312
0.5827649831771851
1.7294700145721436
0.6338905096054077
0.4007173776626587
1.1681199073791504
0.3670607805252075
1.0789848566055298
1.1839946508407593
1.1843847036361694
1.095463752746582
0.4706200957298279
1.036338210105896
0.34247511625289917
0.5407965779304504
0.7143872976303101
0.7786238193511963
0.5346927046775818
0.9798823595046997
1.5010101795196533
0.9740440249443054
0.7524422407150269
0.400176465511322
1.2472015619277954
0.5607710480690002
0.8148735165596008
0.7624585628509521
1.1414839029312134
1.4614002704620361
0.40822896361351013
0.4559836983680725
1.0292688608169556
0.4829498529434204
0.44666868448257446
0.6986138820648193
0.8823777437210083
0.7207658290863037
0.5635466575622559
1.0012590885162354
0.6753945350646973


KeyboardInterrupt: 

In [ ]:
net